<a href="https://colab.research.google.com/github/mtzig/LIDC_GDRO/blob/main/transfer_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
!git clone https://github.com/mtzig/LIDC_GDRO.git
%cd /content/LIDC_GDRO

Cloning into 'LIDC_GDRO'...
remote: Enumerating objects: 2993, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 2993 (delta 35), reused 48 (delta 20), pack-reused 2925
Receiving objects: 100% (2993/2993), 39.03 MiB | 17.40 MiB/s, done.
Resolving deltas: 100% (2847/2847), done.
Checking out files: 100% (5381/5381), done.
/content/LIDC_GDRO


In [47]:
!git pull

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/mtzig/LIDC_GDRO
   e216392..fef34c1  main       -> origin/main
Updating e216392..fef34c1
Fast-forward
 models.py | 5 ++++-
 1 file changed, 4 insertions(+), 1 deletion(-)


In [3]:
import os
import torch
import pandas as pd
import numpy as np
from datasets import NoduleDataset
from dataloaders import SubtypedDataLoader

In [4]:
if torch.cuda.is_available():
    print("Good to go!")
    DEVICE = torch.device("cuda")
else:
    print("Using cpu")
    DEVICE = torch.device("cpu")

Good to go!


In [5]:
lidc_subtyped = pd.read_csv('./data/lidc_spic_subgrouped.csv', index_col=0)
lidc_subtyped.head(5)

,noduleID,malignancy,spiculation,malignancy_b,spiculation_b,subgroup
0,1,5.0,5.0,malignant,marked,marked_malignant
1,2,5.0,1.0,malignant,unmarked,unmarked_malignant
2,3,5.0,2.0,malignant,marked,marked_malignant
3,4,4.0,2.0,malignant,marked,marked_malignant
4,5,5.0,1.0,malignant,unmarked,unmarked_malignant


In [6]:
def getNormed(this_array, this_min = 0, this_max = 255, set_to_int = True):
    
    rat = (this_max - this_min)/(this_array.max() - this_array.min())
    this_array = this_array * rat
    this_array -= this_array.min()
    this_array += this_min
    if set_to_int:
        return this_array.to(dtype= torch.int)
    return this_array

In [7]:
def getImages(image_folder):
    '''
        Input:
        image_folder: directory of the image files

        Output:
        m1: list of the labels encountered (1,2,4,5)
        m2: list of binary labels encountered (benign, malignant)
        diff: list of any nodes with discrepency to CSV labels

    '''
    
    train_img = []
    train_label = []
    
    marked_benign = []
    unmarked_benign = []
    
    marked_malignant = []
    unmarked_malignant = []

    

    lidc = pd.read_csv('./data/lidc_spic_subgrouped.csv')
    train_test = pd.read_csv('./data/lidc_train_test_split_stratified.csv')
    for dir1 in os.listdir(image_folder):
  
        if dir1 == 'Malignancy_3':
            continue

        for file in os.listdir(os.path.join(image_folder, dir1)):


            temp_nodule_ID = file.split('.')[0]
            subtype = lidc[lidc['noduleID']==int(temp_nodule_ID)]['subgroup'].iloc[0]
            malignancy = lidc[lidc['noduleID']==int(temp_nodule_ID)]['malignancy'].iloc[0]
            
            train_type = train_test[train_test['noduleID'] ==int(temp_nodule_ID)]['dataset'].iloc[0]
            
            
            image = np.loadtxt(os.path.join(image_folder, dir1,file))
            image = torch.from_numpy(image).to(DEVICE)
            rgb_image = torch.stack((image,image,image), dim = 0)
            rgb_image = getNormed(rgb_image)
            rgb_image = rgb_image / 255 

            
            
            if train_type == 'train':
                train_img.append(rgb_image)
                train_label.append(torch.tensor(1).to(DEVICE).to(torch.float32) if malignancy > 3 else torch.tensor(0).to(DEVICE).to(torch.float32))
                
                continue
            
            if subtype == 'marked_benign':
                image_array = marked_benign
            elif subtype == 'unmarked_benign':
                image_array = unmarked_benign
            elif subtype == 'marked_malignant':
                image_array = marked_malignant
            else:
                image_array = unmarked_malignant
            
            image_array.append(rgb_image)
 


    return train_img, train_label, marked_benign, unmarked_benign, marked_malignant, unmarked_malignant


In [8]:
train_img, train_label, marked_benign, unmarked_benign, marked_malignant, unmarked_malignant = getImages('./LIDC(MaxSlices)_Nodules(fixed)')

In [9]:
len(train_img)

1219

In [10]:
len(marked_benign)

17

In [11]:
len(unmarked_benign)

149

In [12]:
len(marked_malignant)

83

In [13]:
len(unmarked_malignant)

47

#model train!!!

In [14]:
from models import VGGNet
from loss import ERMLoss
from datasets import NoduleDataset
from dataloaders import InfiniteDataLoader

import train

In [37]:
model = VGGNet(device=DEVICE)

In [38]:
loss_fn = ERMLoss(model,torch.nn.functional.binary_cross_entropy_with_logits,{}) #torch.nn.CrossEntropyLoss()

In [39]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=0.005)


In [18]:
train_dataset = NoduleDataset(train_img, train_label)

In [19]:
len(train_dataset)

1219

In [20]:
train_set, val_set = torch.utils.data.random_split(train_dataset, [1000, 219])

In [42]:
train_loader = InfiniteDataLoader(train_set, 1000, num_workers=0)

In [43]:
val_loader = InfiniteDataLoader(val_set, 219, num_workers = 0)

In [23]:
x,y = next(val_loader)

In [24]:
y.dtype

torch.float32

In [45]:
epochs = 20
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    train.train(train_loader, model, loss_fn, optimizer, verbose=True)
                # train.test(test_dataloader, model)

    model.eval()
    with torch.no_grad():
      X,y = next(val_loader)
      results = torch.sigmoid(model(X))
      accuracy = torch.sum(torch.round(results) == y)/219
    model.train()
    print(f'cv accuracy {accuracy}')

Epoch 1/20
Average training loss: 0.011553945951163769
cv accuracy 0.7260273694992065
Epoch 2/20
Average training loss: 0.013658086769282818
cv accuracy 0.6986300945281982
Epoch 3/20
Average training loss: 0.019219079986214638
cv accuracy 0.6621004343032837
Epoch 4/20
Average training loss: 0.014188161119818687
cv accuracy 0.6986300945281982
Epoch 5/20
Average training loss: 0.013350579887628555
cv accuracy 0.6986300945281982
Epoch 6/20
Average training loss: 0.021042002364993095
cv accuracy 0.6849314570426941
Epoch 7/20
Average training loss: 0.014486176893115044
cv accuracy 0.680365264415741
Epoch 8/20
Average training loss: 0.008896777406334877
cv accuracy 0.7031963467597961
Epoch 9/20
Average training loss: 0.008173934184014797
cv accuracy 0.7077625393867493
Epoch 10/20
Average training loss: 0.007595770992338657
cv accuracy 0.6757990717887878
Epoch 11/20
Average training loss: 0.010732823051512241
cv accuracy 0.6621004343032837
Epoch 12/20
Average training loss: 0.0109863607212901